In [ ]:
# Yonghwan Kim Assignment3 Task2 simple search engine-1 
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 27.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6f78012332fa23923778b17318966c3552ac7cc4b0ffaa5e08989cd785d105a3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import sys
import re
import math
from pyspark import SparkContext, SparkConf

In [ ]:
# create Spark context with necessary configuration
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('Task2')
sc = SparkContext(conf=conf)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Give **.txt FILE PATHS** here
# You can use any random input file of your choice
file2Dir = '/content/drive/MyDrive/Assignment3_cloud/Assignment3_Task2Data.txt'
stopwordDir = '/content/drive/MyDrive/Assignment3_cloud/stopwords-en.txt'
fileSaveDir = '/content/drive/MyDrive/Assignment3_cloud/TF_index'
# USE THIS FILE as input FOR ALL BELOW tasks
# Change filepath to HDFS path when executing in the cluster

In [ ]:
# broadcast stopwords
broadcastStopword = sc.broadcast(list(open(stopwordDir).read().splitlines()))
# bring data to rdd
fileRDD = sc.textFile(file2Dir)

# step 1: Text processing

# case folding
casefoldingRDD = fileRDD.map(lambda line: line.lower())
# change formatting
linesRDD = casefoldingRDD.map(lambda line: line.replace("<text>", "").split("</text>"))
formatRDD = linesRDD.map(lambda line: line[0].replace("<docid>", "").split("</docid>"))
# remove urls
URL_Kill = re.compile(r'(https?:|www.)\S+')
URLRemoverRDD = formatRDD.map(lambda word: (word[0], re.sub(URL_Kill, " ", word[1])))
# remove all non-alphabetic chracters and words
punctuationsRemovalRDD = URLRemoverRDD.map(lambda word: (word[0], re.sub(r'[^a-zA-Z ]', ' ', word[1]).split(" ")))
# remove empty elements
emptyRemoveRDD = punctuationsRemovalRDD.map(lambda word: (word[0], [i for i in word[1] if i]))
# remove stopwords
stopwordsRemoval = emptyRemoveRDD.map(lambda word: (word[0], [i for i in word[1] if i not in broadcastStopword.value]))

In [ ]:
# Step-2: Term Frequency index construction

# constructing termfrequecy table
termFreqTableRDD1 = stopwordsRemoval.flatMap(lambda word: [((w, word[0]), 1) for w in word[1]])
# combine by key
countRDD = termFreqTableRDD1.reduceByKey(lambda a,b: a+b)
# calculate freq with the log-weigthed frequency
termFreqTableRDD2 = countRDD.map(lambda t: (t[0][0], (t[0][1], 1 + math.log10(t[1]))))
# group by key
termFreqTableRDD3 = termFreqTableRDD2.groupByKey()
# covert to list to match the given format
termFreqTableRDD4 = termFreqTableRDD3.map(lambda l: (l[0], list(l[1])))
# change dictionary to string format with the given format
rddtostringRDD = termFreqTableRDD4.map(lambda l: l[0] + "@" + '+'.join(str(e[0]) + "#" + str(e[1]) for e in l[1]))
# save the rdd to the TF_index directory
rddtostringRDD.saveAsTextFile(fileSaveDir)